### Import Libraries

In [1]:
from pydantic import BaseModel,Field
from typing import List
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import os

C:\Users\gadda\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Output Parsing

In [2]:
class Resume(BaseModel):
        name: str = Field(description="Full name of the candidate")
        email: str = Field(description="Email address")
        phone: str = Field(description="Phone number if available")
        skills: List[str] = Field(description="List of technical skills")
        experience: List[str] = Field(description="Work experience details")
        education: List[str] = Field(description="Educational background")

### Prompt Template

In [3]:
parser=PydanticOutputParser(pydantic_object=Resume)

prompt=PromptTemplate(

    input_variables=['resume_text'],
    
    template=""" Your a expert resume parser
    Extract the structured information from the resume given below 
    Follow the format instructions give below
    Do not hallucinate missing fields
    If information is not present ,please give back empty string or empty list
    IMPORTANT RULES:
    - Skills must be SHORT, NORMALIZED skill names (1–3 words max)
    - Do NOT include full sentences in skills
    - Infer skills conservatively from the resume text
    - Do NOT hallucinate
    - If information is missing, return empty values
    {format_instructions}

    Resume_text:
    {resume_text}
                    """,
    partial_variables={ 'format_instructions':parser.get_format_instructions()

    }
    
)

### Initialize LLM

In [4]:
LLM=ChatMistralAI(model_name='mistral-small',
                 api_key=os.getenv('MISTRAL_API_KEY'),
                 temperature=0.0) 

In [5]:
with open(r"C:\Users\gadda\OneDrive\Desktop\AI_Resume_Parser\Data\resume.txt",'r',encoding='utf-8') as f:
    resume_text=f.read()
resume_text[:5000]    

"Hayden Smith  \n214 Mitre Avenue, Park Hill, 3045 |  04501 123 456  |  haydensmith@email.com \nPage 1 \n \n \n(Tip: Name, mobile number and email address are essential. Current address could be included \nespecially if you live nearby. Ensure that it is clearly displayed) \nCareer Objective  \nI am reliable hard working Year 11 student seeking casual or part-time customer service work in a \nsports retail environment. Having played soccer for nine years and a keen all-round sports \nenthusiast, I am looking to contribute knowledge and proven communications skills.  \n \n(Tip: A career objective isn’t essential, but it’s useful if you don't have much experience and can \nconvey enthusiasm and motivation. Briefly summarise any work you have done, your strengths and \nrelevant expertise and state how you aim to apply this to your career goal. Adjust the statement to \nreflect the role you are applying for.)   \nAvailability \nMonday – Friday: 4.30pm – 10.00pm \nSaturday – Sunday: 8.00am 

### Intialize Chains

In [6]:
chains=prompt | LLM | parser
structured_resume=chains.invoke({
    'resume_text':resume_text
})
structured_resume=structured_resume.model_dump()

In [7]:
structured_resume

{'name': 'Hayden Smith',
 'email': 'haydensmith@email.com',
 'phone': '04501 123 456',
 'skills': ['Customer service',
  'Cash handling',
  'Communication',
  'Teamwork',
  'Organization',
  'Problem solving'],
 'experience': ['Customer service (volunteer) at Park Hill Soccer Club Canteen (December 2016 – March 2017)',
  'Newspaper deliverer at Argo Newsagency (June 2016 – February 2017)',
  'Soccer umpire for under 14 team at Park Hill Soccer Club (2016 – current)',
  'Assistant Coach for junior players at Park Hill Soccer Club (2017 – current)'],
 'education': ['Year 11 at Park Hill Secondary College (Subjects: Maths, English, Business Management, VET studies in Sport and Recreation)']}

In [12]:
import json, os



with open("../data/processed/structured_resume.json", "w", encoding="utf-8") as f:
    json.dump(structured_resume, f, indent=2)
